In this project I work on a file of metropolitan regions and associated sports teams.

For each sport I would like to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? (Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses.)

I would also like to explore the hypothesis that given that an area has two sports teams in different sports, those teams will perform the same within their respective sports. I will do this with a series of paired t-tests between all pairs of sports.

I'm using data from year 2018 for my analysis.

Analysis is from the perspective of the metropolitan region (teams which are commonly known by a different area (e.g. "Oakland Raiders") will be mapped into the metropolitan region given (e.g. San Francisco Bay Area)).

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import lxml
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
nhl_df=pd.read_csv("data/american_sport_teams/nhl.csv")
cities=pd.read_html("data/american_sport_teams/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# extracting NHL team names from cities
cities["NHL"] = cities["NHL"].str.strip().str.extract("(^[A-Z\d]+[A-Za-z\s]*)")
cities = cities[cities["NHL"].notna()]

cities.rename(columns={"Population (2016 est.)[8]":"Population"}, inplace=True)
cities = cities.sort_values(by="Metropolitan area")

nhl_df = nhl_df[nhl_df["year"]==2018]
nhl_df = nhl_df[nhl_df["GP"].str.isnumeric()]
nhl_df = nhl_df.iloc[:, [0,2,3]]

# extracting NHL team names from nhl_df
nhl_df["team"]=nhl_df["team"].str.strip().str.extract("[A-Z]{1}[a-z.\s]+([A-Za-z\s\d]+[a-z]+)")
nhl_df["team"][nhl_df["team"].str.contains("Lightning")]="Lightning"
nhl_df["team"][nhl_df["team"].str.contains("Devils")]="Devils"
nhl_df["team"][nhl_df["team"].str.contains("Islanders")]="Islanders"
nhl_df["team"][nhl_df["team"].str.contains("Rangers")]="Rangers"
nhl_df["team"][nhl_df["team"].str.contains("Blues")]="Blues"
nhl_df["team"][nhl_df["team"].str.contains("Sharks")]="Sharks"
nhl_df["team"][nhl_df["team"].str.contains("Kings")]="Kings"

# calulating win/loss ratio
nhl_df["win/loss ratio"]=nhl_df["W"].astype(int)/(nhl_df["W"].astype(int)+nhl_df["L"].astype(int))

#merging cities with nhl_df on team names
nhl_df = pd.merge(nhl_df, cities, left_on="team", right_on="NHL", how="outer")

nhl_df["Metropolitan area"][nhl_df["team"]=="Rangers"]="New York City"
nhl_df["Metropolitan area"][nhl_df["team"]=="Islanders"]="New York City"
nhl_df["Metropolitan area"][nhl_df["team"]=="Devils"]="New York City"
nhl_df["Metropolitan area"][nhl_df["team"]=="Kings"]="Los Angeles"
nhl_df["Metropolitan area"][nhl_df["team"]=="Ducks"]="Los Angeles"
nhl_df.drop([31,32], inplace=True)

nhl_df= nhl_df.groupby("Metropolitan area")["win/loss ratio"].mean().sort_index()
nhl_df.name="NHL"

# calculating hockey teams win/loss ratio's correlation with the population of the city it is in
def nhl_correlation():    
    population_by_region = cities["Population"].astype(int)
    return stats.pearsonr(population_by_region, nhl_df)

corr_nhl, p_val_nhl = nhl_correlation()
corr_nhl

0.012486162921209923

In [3]:
nba_df=pd.read_csv("data/american_sport_teams/nba.csv")
cities=pd.read_html("data/american_sport_teams/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# extracting NBA team names from cities
cities["NBA"] = cities["NBA"].str.strip().str.extract("(^[A-Z\d]+[A-Za-z\s]*)")
cities = cities[cities["NBA"].notna()]

cities.rename(columns={"Population (2016 est.)[8]":"Population"}, inplace=True)
cities = cities.sort_values(by="Metropolitan area")

nba_df = nba_df[nba_df["year"]==2018]
nba_df = nba_df.iloc[:, [0,1,2]]

# extracting NBA team names from nba_df
nba_df["team"]=nba_df["team"].str.strip().str.extract("[A-Z]{1}[a-z.\s]+([A-Za-z\s\d]+[a-z]+)")
nba_df["team"][nba_df["team"].str.contains("Knicks")]="Knicks"
nba_df["team"][nba_df["team"].str.contains("Warriors")]="Warriors"
nba_df["team"][nba_df["team"].str.contains("Thunder")]="Thunder"
nba_df["team"][nba_df["team"].str.contains("Pelicans")]="Pelicans"
nba_df["team"][nba_df["team"].str.contains("Spurs")]="Spurs"
nba_df["team"][nba_df["team"].str.contains("Clippers")]="Clippers"
nba_df["team"][nba_df["team"].str.contains("Lakers")]="Lakers"

# calulating win/loss ratio
nba_df["win/loss ratio"]=nba_df["W"].astype(int)/(nba_df["W"].astype(int)+nba_df["L"].astype(int))

# merging cities with nba_df on team names
nba_df = pd.merge(nba_df, cities, left_on="team", right_on="NBA", how="outer")

nba_df["Metropolitan area"][nba_df["team"]=="Knicks"]="New York City"
nba_df["Metropolitan area"][nba_df["team"]=="Nets"]="New York City"
nba_df["Metropolitan area"][nba_df["team"]=="Lakers"]="Los Angeles"
nba_df["Metropolitan area"][nba_df["team"]=="Clippers"]="Los Angeles"
nba_df.drop([30,31], inplace=True)

nba_df = nba_df.groupby("Metropolitan area")["win/loss ratio"].mean().sort_index()
nba_df.name="NBA"

# calculating basketball teams win/loss ratio's correlation with the population of the city it is in
def nba_correlation():  
    population_by_region = cities["Population"].astype(int)
    return stats.pearsonr(population_by_region, nba_df)

corr_nba, p_val_nba = nba_correlation()
corr_nba

-0.17657160252844623

In [4]:
mlb_df=pd.read_csv("data/american_sport_teams/mlb.csv")
cities=pd.read_html("data/american_sport_teams/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# extracting MLB team names from cities
cities["MLB"] = cities["MLB"].str.strip().str.extract("(^[A-Z\d]+[A-Za-z\s]*)")
cities = cities[cities["MLB"].notna()]
cities.rename(columns={"Population (2016 est.)[8]":"Population"}, inplace=True)
cities = cities.sort_values(by="Metropolitan area")

mlb_df = mlb_df[mlb_df["year"]==2018]
mlb_df = mlb_df.iloc[:, [0,1,2]]

# extracting MLB team names from mlb_df
mlb_df["team"]=mlb_df["team"].str.strip().str.extract("[A-Z]{1}[a-z.\s]+([A-Za-z\s\d]+[a-z]+)")
mlb_df["team"][mlb_df["team"].str.contains("Yankees")]="Yankees"
mlb_df["team"][mlb_df["team"].str.contains("Rays")]="Rays"
mlb_df["team"][mlb_df["team"].str.contains("Royals")]="Royals"
mlb_df["team"][mlb_df["team"].str.contains("Angels")]="Angels"
mlb_df["team"][mlb_df["team"].str.contains("Mets")]="Mets"
mlb_df["team"][mlb_df["team"].str.contains("Cardinals")]="Cardinals"
mlb_df["team"][mlb_df["team"].str.contains("Dodgers")]="Dodgers"
mlb_df["team"][mlb_df["team"].str.contains("Giants")]="Giants"
mlb_df["team"][mlb_df["team"].str.contains("Padres")]="Padres"

# calulating win/loss ratio
mlb_df["win/loss ratio"]=mlb_df["W"].astype(int)/(mlb_df["W"].astype(int)+mlb_df["L"].astype(int))

#merging cities with nba_df on team names
mlb_df = pd.merge(mlb_df, cities.iloc[:,[0,3]], left_on="team", right_on="MLB", how="outer")

mlb_df["Metropolitan area"][mlb_df["team"]=="Yankees"]="New York City"
mlb_df["Metropolitan area"][mlb_df["team"]=="Mets"]="New York City"
mlb_df["Metropolitan area"][mlb_df["team"]=="Dodgers"]="Los Angeles"
mlb_df["Metropolitan area"][mlb_df["team"]=="Angels"]="Los Angeles"
mlb_df["Metropolitan area"][mlb_df["team"]=="Cubs"]="Chicago"
mlb_df["Metropolitan area"][mlb_df["team"]=="White Sox"]="Chicago"
mlb_df["Metropolitan area"][mlb_df["team"]=="Giants"]="San Francisco Bay Area"
mlb_df["Metropolitan area"][mlb_df["team"]=="Athletics"]="San Francisco Bay Area"

mlb_df.drop([30,31,32,33], inplace=True)
mlb_df = mlb_df.groupby("Metropolitan area")["win/loss ratio"].mean().sort_index()
mlb_df.name="MLB"

# calculating baseball teams win/loss ratio's correlation with the population of the city it is in
def mlb_correlation(): 
    population_by_region = cities["Population"].astype(int)
    return stats.pearsonr(population_by_region, mlb_df)

corr_mlb, p_val_mlb = mlb_correlation()
corr_mlb


0.15027698302669307

In [5]:
nfl_df=pd.read_csv("data/american_sport_teams/nfl.csv")
cities=pd.read_html("data/american_sport_teams/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nfl_df=nfl_df[nfl_df["year"]==2018]

# extracting NFL team names from cities
cities["NFL"] = cities["NFL"].str.strip().str.extract("(^[A-Z\d]+[A-Za-z\s]*)")
cities = cities[cities["NFL"].notna()]

cities.rename(columns={"Population (2016 est.)[8]":"Population"}, inplace=True)
cities = cities.sort_values(by="Metropolitan area")

nfl_df = nfl_df.loc[:, ["team", "W", "L"]]
nfl_df = nfl_df[nfl_df["L"].str.isnumeric()]

# extracting NFL team names from nfl_df
nfl_df["team"]=nfl_df["team"].str.strip().str.extract("[A-Z]{1}[a-z.\s]+([A-Za-z\s\d]+[a-z]+)")
nfl_df["team"][nfl_df["team"].str.contains("Patriots")]="Patriots"
nfl_df["team"][nfl_df["team"].str.contains("Jets")]="Jets"
nfl_df["team"][nfl_df["team"].str.contains("Chiefs")]="Chiefs"
nfl_df["team"][nfl_df["team"].str.contains("Chargers")]="Chargers"
nfl_df["team"][nfl_df["team"].str.contains("Giants")]="Giants"
nfl_df["team"][nfl_df["team"].str.contains("Packers")]="Packers"
nfl_df["team"][nfl_df["team"].str.contains("Saints")]="Saints"
nfl_df["team"][nfl_df["team"].str.contains("Buccaneers")]="Buccaneers"
nfl_df["team"][nfl_df["team"].str.contains("Rams")]="Rams"
nfl_df["team"][nfl_df["team"].str.contains("49ers")]="49ers"

# calulating win/loss ratio
nfl_df["win/loss ratio"]=nfl_df["W"].astype(int)/(nfl_df["W"].astype(int)+nfl_df["L"].astype(int))

# merging cities with nfl_df on team names
nfl_df = pd.merge(nfl_df, cities, left_on="team", right_on="NFL", how="outer")

nfl_df["Metropolitan area"][nfl_df["team"]=="Giants"]="New York City"
nfl_df["Metropolitan area"][nfl_df["team"]=="Jets"]="New York City"
nfl_df["Metropolitan area"][nfl_df["team"]=="Rams"]="Los Angeles"
nfl_df["Metropolitan area"][nfl_df["team"]=="Chargers"]="Los Angeles"
nfl_df["Metropolitan area"][nfl_df["team"]=="49ers"]="San Francisco Bay Area"
nfl_df["Metropolitan area"][nfl_df["team"]=="Raiders"]="San Francisco Bay Area"
nfl_df.drop([32,33,34], inplace=True)

nfl_df = nfl_df.groupby("Metropolitan area")["win/loss ratio"].mean().sort_index()
nfl_df.name="NFL"

# calculating football teams win/loss ratio's correlation with the population of the city it is in
def nfl_correlation(): 
    population_by_region = cities["Population"].astype(int) 
    return stats.pearsonr(population_by_region, nfl_df)

corr_nfl, p_val_nfl = nfl_correlation()
corr_nfl



0.004922112149349409

In [6]:
cities=pd.read_html("data/american_sport_teams/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,5,6,7,8]]

# extracting sport team names from cities
cities["NFL"] = cities["NFL"].str.strip().str.extract("(^[A-Z\d]+[A-Za-z\s]*)")
cities["MLB"] = cities["MLB"].str.strip().str.extract("(^[A-Z\d]+[A-Za-z\s]*)")
cities["NBA"] = cities["NBA"].str.strip().str.extract("(^[A-Z\d]+[A-Za-z\s]*)")
cities["NHL"] = cities["NHL"].str.strip().str.extract("(^[A-Z\d]+[A-Za-z\s]*)")

big4 = [mlb_df, nfl_df, nhl_df, nba_df]


def sports_team_performance():
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    # creating empty data frame with sport names in rows and columns
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for i in range(3):
        j=i+1
        while j<4:
            # merging sports win/loss ratio on metropolitan area
            sports_merged=pd.merge(big4[i], big4[j], left_index=True, right_index=True)
            t_stat, p_val = stats.ttest_rel(sports_merged.iloc[:,0], sports_merged.iloc[:,1])
            # adding p-values to data frame
            p_values.loc[big4[i].name,big4[j].name]=p_val
            p_values.loc[big4[j].name,big4[i].name]=p_val
            j+=1
    return p_values

p_values=sports_team_performance()
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN


As we can see abowe, we can reject the null hypothesis (at the 0.05 significance level) that given that an area has two sports teams in different sports, those teams will perform the same within their respective sports for three pairs of sports: (NHL, NFL), (NHL, NBA) and (NHL, MLB).